<center class="mytitle">
**GT Numérique**
</center>

# Méthodes de Schwarz: quelques idées d'implémentation

<center>
<span>**Loic Gouarin**</span>
</center>
<center>
<span>12 décembre 2017</span>
</center>

### Plan

- Schwarz classique appliqué au problème de Poisson 1D
- Construction d'un préconditionneur "BDD" pour le problème d'élasticité linéaire 2D

## Schwarz classique pour le problème de Poisson 1D

### Problème étudié

On souhaite résoudre 

$$
\left\{
\begin{array}{l}
-\Delta u(x) = 1 \; \text{pour} \; x\in[0, 1], \\
u(0)= u(1) = 0.
\end{array}
\right.
$$

à l'aide d'un schéma aux différences finies.

### Initialisation du problème

In [1]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as linsolve

In [2]:
N = 101
L = 1.
dx = 1./(N - 1)
x = np.arange(0, L+.5*dx, dx)

### Construction de la matrice et du second membre

In [3]:
A = sp.spdiags([-np.ones(N), 2*np.ones(N), -np.ones(N)],[-1, 0, 1], N, N).tocsc()
b = np.ones(N)*dx**2

On applique les conditions de bords

In [4]:
A[0 ,  0] = 1; A[ 0,  1] = 0
A[-1, -2] = 0; A[-1, -1] = 1

In [5]:
b[0] = 0; b[-1] = 0

### Résolution du système

In [6]:
LU = linsolve.factorized(A)

In [7]:
u = LU(b)

### Représentation de la solution

In [8]:
from bqplot import *
x_sc, y_sc = LinearScale(), LinearScale()

ax_x = Axis(label='x', scale=x_sc, grid_lines='solid')
ax_y = Axis(label='solution', scale=y_sc, orientation='vertical', grid_lines='solid')

In [9]:
line = Lines(x=x, y=u, scales={'x': x_sc, 'y': y_sc})

In [10]:
fig = Figure(axes=[ax_x, ax_y], marks=[line])
fig

Figure(axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='solution', orientation='vertical', scale=LinearScale(), side='left')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.  ,  0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,
        0.09,  0.1 ,  0.11,  0.12,  0.13,  0.14,  0.15,  0.16,  0.17,
        0.18,  0.19,  0.2 ,  0.21,  0.22,  0.23,  0.24,  0.25,  0.26,
        0.27,  0.28,  0.29,  0.3 ,  0.31,  0.32,  0.33,  0.34,  0.35,
        0.36,  0.37,  0.38,  0.39,  0.4 ,  0.41,  0.42,  0.43,  0.44,
        0.45,  0.46,  0.47,  0.48,  0.49,  0.5 ,  0.51,  0.52,  0.53,
        0.54,  0.55,  0.56,  0.57,  0.58,  0.59,  0.6 ,  0.61,  0.62,
        0.63,  0.64,  0.65,  0.66,  0.67,  0.68,  0.69,  0.7 ,  0.71,
        0.72,  0.73,  0.74,  0.75,  0.76,  0.77,  0.78,  0.79,  0.8 ,
        0.81,  0.82,  0.83,  0.84,  0.85,  0.86,  0.87,  0.88,  0.89,
        0.9 ,  0.91,  0.92,  0.93,  0.94,  0.95,  0.96,  0.97,  0.98,
        0.99,  1.  ]), y=array([ 0.     ,  0.00495,  0.0098 ,  0.01455,  0.0192 ,  0.02375,
        0.0282 ,  0.03255,  0.0368 ,  0.04095,  0.045  ,  0.04895,
        0.0528 ,  0.05655,  0.0602 ,  0.06375,  0.0672 ,  0.07055,
        0.0738 ,  0.07695,  0.08   ,  0.08295,  0.0858 ,  0.08855,
        0.0912 ,  0.09375,  0.0962 ,  0.09855,  0.1008 ,  0.10295,
        0.105  ,  0.10695,  0.1088 ,  0.11055,  0.1122 ,  0.11375,
        0.1152 ,  0.11655,  0.1178 ,  0.11895,  0.12   ,  0.12095,
        0.1218 ,  0.12255,  0.1232 ,  0.12375,  0.1242 ,  0.12455,
        0.1248 ,  0.12495,  0.125  ,  0.12495,  0.1248 ,  0.12455,
        0.1242 ,  0.12375,  0.1232 ,  0.12255,  0.1218 ,  0.12095,
        0.12   ,  0.11895,  0.1178 ,  0.11655,  0.1152 ,  0.11375,
        0.1122 ,  0.11055,  0.1088 ,  0.10695,  0.105  ,  0.10295,
        0.1008 ,  0.09855,  0.0962 ,  0.09375,  0.0912 ,  0.08855,
        0.0858 ,  0.08295,  0.08   ,  0.07695,  0.0738 ,  0.07055,
        0.0672 ,  0.06375,  0.0602 ,  0.05655,  0.0528 ,  0.04895,
        0.045  ,  0.04095,  0.0368 ,  0.03255,  0.0282 ,  0.02375,
        0.0192 ,  0.01455,  0.0098 ,  0.00495,  0.     ]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))

### Schwarz alterné (Schwarz 1870)

<center>
<img style="width:35%;" src="./figures/schwarz.png"/>
</center>

<table>
<tr>
<td>
$$
\scriptsize{
\left\{
\begin{array}{l}
\mathcal{L}(u_1^{k+1}) = f(x), \; x \in \Omega_1 \\
u_1^{k+1}(x) = g(x), \; x\in \partial \Omega_1 \bigcap \overline \Omega_1 \\
u_1^{k+1}(x) = u_2^k(x), \; x \in \Gamma_1
\end{array}
\right.}
$$
    </td>
<td>
$$
\scriptsize{
\left\{
\begin{array}{l}
\mathcal{L}(u_2^{k+1}) = f(x), \; x \in \Omega_2 \\
u_2^{k+1}(x) = g(x), \; x\in \partial \Omega_2 \bigcap \overline \Omega_2 \\
u_2^{k+1}(x) = u_1^{k+1}(x), \; x\in \Gamma_2
\end{array}
\right.
}
$$
    </td>
    </tr>
</table>

### Schwarz parallèle (Lions 1988)

<center>
<img style="width:35%;" src="./figures/schwarz.png"/>
</center>

<table>
<tr>
<td>
$$
\scriptsize{
\left\{
\begin{array}{l}
\mathcal{L}(u_1^{k+1}) = f(x), \; x \in \Omega_1 \\
u_1^{k+1}(x) = g(x), \; x\in \partial \Omega_1 \bigcap \overline \Omega_1 \\
u_1^{k+1}(x) = u_2^k(x), \; x \in \Gamma_1
\end{array}
\right.}
$$
    </td>
<td>
$$
\scriptsize{
\left\{
\begin{array}{l}
\mathcal{L}(u_2^{k+1}) = f(x), \; x \in \Omega_2 \\
u_2^{k+1}(x) = g(x), \; x\in \partial \Omega_2 \bigcap \overline \Omega_2 \\
u_2^{k+1}(x) = u_1^{k}(x), \; x\in \Gamma_2
\end{array}
\right.
}
$$
    </td>
    </tr>
</table>

### Utilisation du parallèlisme dans les notebooks

*Attention:* avant d'exécuter les lignes suivantes, il faut démarrer les processus en allant à la page qui se trouve sur la page de demarrage des notebooks dans l'onglet `IPython cluster`. Nous en démarrerons 4 pour la suite.

In [11]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')

In [12]:
view = c[:]
view.activate()
view.block = True

### mpi4py

`mpi4py` est une implémentation de MPI en Python.

MPI permet d'envoyer des messages entre processus.

Les types de message

- communications point à point
- communications collectives

### mpi4py

Nous aurons besoin dans la suite

- `COMM_WORLD`: communicateur englobant l'ensemble des processus initialisés.
- `size`: le nombre de processus
- `rank`: le numéro du processus
- `send` et `recv`: envoi et réception de valeurs

### Initialisation des processus

In [13]:
%%px
import mpi4py.MPI as mpi
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as linsolve

size = mpi.COMM_WORLD.size
rank = mpi.COMM_WORLD.rank

N = 101
L = 1.
dx = 1./(N - 1)

### Découpage du domaine en sous domaines

In [14]:
%%px
Nl = (N-1)//size + 1
if mpi.COMM_WORLD.rank == size-1:
    Nl += (N-1)%size
    
Narray = mpi.COMM_WORLD.allgather(Nl-1)
tmp = np.cumsum(Narray)
tmp = np.insert(tmp, 0, 0)
tmp

Out[0:22]: array([  0,  25,  50,  75, 100])

Out[1:22]: array([  0,  25,  50,  75, 100])

Out[2:22]: array([  0,  25,  50,  75, 100])

Out[3:22]: array([  0,  25,  50,  75, 100])

### Découpage du domaine en sous domaines

In [15]:
%%px
beg = tmp[rank]*dx
end = tmp[rank+1]*dx

overlap = 2

if rank > 0:
    beg -= dx*overlap
if rank < size - 1:
    end += dx*overlap

xl = np.arange(beg, end + .5*dx, dx)

### Construction des matrices locales

In [16]:
%%px
n = xl.size

A = sp.spdiags([-np.ones(n), 2*np.ones(n), -np.ones(n)],[-1, 0, 1], n, n) # 1D poisson matrix
A = A.tocsr()

A[0 ,  0] = 1; A[ 0,  1] = 0;
A[-1, -2] = 0; A[-1, -1] = 1;

b = np.ones(n)*dx**2
b[0] = 0; b[-1] = 0

### Résolution des problèmes locaux

In [17]:
%%px 
LU = linsolve.factorized(A.tocsc())
u = LU(b)

### Algorithme de Schwarz parallèle

<center>
<img style="width:90%;" src="./figures/schwarz_dd.png"/>
 </center>

### Algorithme de Schwarz parallèle

In [18]:
def update(nbite=100):
    U = []
    U.append(mpi.COMM_WORLD.allgather(u))
    X = mpi.COMM_WORLD.allgather(xl)
    for k in range(nbite):
        if rank == 0:
            mpi.COMM_WORLD.send(u[-1-2*overlap], rank + 1, rank)
            b[-1] = mpi.COMM_WORLD.recv(None, rank + 1, rank + 1)
        elif rank == size - 1:
            mpi.COMM_WORLD.send(u[2*overlap], rank - 1, rank)
            b[0] = mpi.COMM_WORLD.recv(None, rank - 1, rank - 1)
        else:
            mpi.COMM_WORLD.send(u[-1-2*overlap], rank + 1, rank)
            b[-1] = mpi.COMM_WORLD.recv(None, rank + 1, rank + 1)
            mpi.COMM_WORLD.send(u[2*overlap], rank - 1, rank)
            b[0] = mpi.COMM_WORLD.recv(None, rank - 1, rank - 1)
        u[:] = LU(b)
        U.append(mpi.COMM_WORLD.allgather(u))
    return X, U

In [19]:
from classical_schwarz import plot_solution
plot_solution(view)

interactive(children=(IntSlider(value=0, continuous_update=False, description='iteration'), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=1, continuous_update=False, description='overlap', max=20), Output()), _dom_classes=('widget-interact',))

VBox(children=(HBox(children=(Play(value=0, description='Press play'), IntSlider(value=0, continuous_update=False, description='iteration'), IntSlider(value=1, continuous_update=False, description='overlap', max=20))), Figure(animation_duration=100, axes=[Axis(label='x', scale=LinearScale()), Axis(label='solution', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.  ,  0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,
        0.09,  0.1 ,  0.11,  0.12,  0.13,  0.14,  0.15,  0.16,  0.17,
        0.18,  0.19,  0.2 ,  0.21,  0.22,  0.23,  0.24,  0.25,  0.26]), y=array([ 0.     ,  0.00125,  0.0024 ,  0.00345,  0.0044 ,  0.00525,
        0.006  ,  0.00665,  0.0072 ,  0.00765,  0.008  ,  0.00825,
        0.0084 ,  0.00845,  0.0084 ,  0.00825,  0.008  ,  0.00765,
        0.0072 ,  0.00665,  0.006  ,  0.00525,  0.0044 ,  0.00345,
        0.0024 ,  0.00125,  0.     ])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.24,  0.25,  0.26,  0.27,  0.28,  0.29,  0.3 ,  0.31,  0.32,
        0.33,  0.34,  0.35,  0.36,  0.37,  0.38,  0.39,  0.4 ,  0.41,
        0.42,  0.43,  0.44,  0.45,  0.46,  0.47,  0.48,  0.49,  0.5 ,  0.51]), y=array([ 0.    ,  0.0013,  0.0025,  0.0036,  0.0046,  0.0055,  0.0063,
        0.007 ,  0.0076,  0.0081,  0.0085,  0.0088,  0.009 ,  0.0091,
        0.0091,  0.009 ,  0.0088,  0.0085,  0.0081,  0.0076,  0.007 ,
        0.0063,  0.0055,  0.0046,  0.0036,  0.0025,  0.0013,  0.    ])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.49,  0.5 ,  0.51,  0.52,  0.53,  0.54,  0.55,  0.56,  0.57,
        0.58,  0.59,  0.6 ,  0.61,  0.62,  0.63,  0.64,  0.65,  0.66,
        0.67,  0.68,  0.69,  0.7 ,  0.71,  0.72,  0.73,  0.74,  0.75,  0.76]), y=array([ 0.    ,  0.0013,  0.0025,  0.0036,  0.0046,  0.0055,  0.0063,
        0.007 ,  0.0076,  0.0081,  0.0085,  0.0088,  0.009 ,  0.0091,
        0.0091,  0.009 ,  0.0088,  0.0085,  0.0081,  0.0076,  0.007 ,
        0.0063,  0.0055,  0.0046,  0.0036,  0.0025,  0.0013,  0.    ])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.74,  0.75,  0.76,  0.77,  0.78,  0.79,  0.8 ,  0.81,  0.82,
        0.83,  0.84,  0.85,  0.86,  0.87,  0.88,  0.89,  0.9 ,  0.91,
        0.92,  0.93,  0.94,  0.95,  0.96,  0.97,  0.98,  0.99,  1.  ]), y=array([ 0.     ,  0.00125,  0.0024 ,  0.00345,  0.0044 ,  0.00525,
        0.006  ,  0.00665,  0.0072 ,  0.00765,  0.008  ,  0.00825,
        0.0084 ,  0.00845,  0.0084 ,  0.00825,  0.008 

## Problème d'élasticité linéaire 2D

### Description du problème

<center>
<img style="width:80%;" src="./figures/poutre.png"/>
</center>

### Description du problème

Nous considérons ici un corps élastique qui occupe, dans sa configuration de référence, un domaine borné $\Omega$ dans $\mathbb{R}^2$ avec un bord noté $\Gamma$.

Soit $\left\{ \Gamma_D, \Gamma_N\right\}$ une partition de $\Gamma$ où $\Gamma_N$ peut être vide.

Nous aurons

- des conditions de Dirichlet sur $\Gamma_D$,
- des conditions de Neumann sur $\Gamma_N$.

Soit $u=(u_1, u_2)$ le champs des déplacements de ce corps élastique.

### Description du problème

Sous l'hypothèse de petites déformations, les tenseurs de contraintes et de déformations s'écrivent

$$
\sigma_{ij}(u) = 2\mu\epsilon_{ij} + \lambda tr(\epsilon(u))\mathbb{1}_2, \; i,j=1,2,\\
$$
$$
\epsilon(u) = \frac{\nabla u+ \nabla u^T}{2},
$$

où $\lambda$ et $\mu$ sont les constantes de Lamé. Ces coefficients peuvent être calculés à partir du module d'Young $E$ et le coefficient de Poisson $\nu$

$$
\lambda = \frac{\nu E}{(1+\nu)(1-2\nu)}, \; \mu = \frac{E}{2(1+\nu)}.
$$

### Description du problème

En se donnant $f=\left(f_1, f_2\right)\in L^2(\Omega)$, $g=(g_1, g_2)\in L^2(\Gamma)$ et $u_D$, le problème étudié s'écrit

$$
\begin{array}{rll}
-div\sigma(u) &=& f, \; \text{dans} \; \Omega,\\
\sigma(u)\cdot n &=& g, \; \text{sur} \; \Gamma_N, \\
u &=& u_D,\; \text{sur} \; \Gamma_D. 
\end{array}
$$

### Description du problème

Soient

$$
\begin{array}{rll}
V &=& \left\{ v \in H^1(\Omega) \; : \; v=0 \; \text{sur} \; \Gamma_D \right\},\\
V^D &=& \left\{ v \in H^1(\Omega) \; : \; v=u_D \; \text{sur} \; \Gamma_D \right\}.
\end{array}
$$

### Description du problème
La formulation variationnelle du problème d'élasticité s'écrit

Trouver $u\in V^D$ tel que 

$$
\int_\Omega \epsilon(u):\mathbb{C}\epsilon(v)dx=\int_\Omega f\cdot v dx + \int_{\Gamma_N} g\cdot v ds, \; \forall v \in V.
$$

avec

$$
\mathbb{C}_{ijkl} = \lambda \delta_{ij}\delta_{kl} + \mu (\delta_{ik}\delta_{jl}+\delta_{il}\delta_{jk}), \; 1\leq i,j,k,l\leq 2,
$$

où $\delta_{ij}$ est le symbôle de Kronecker.

### Discrétisation éléments finis

Soit $\mathcal{Q}_h$ une quadrangulation de $\Omega$. Les espaces $V$ et $ V^D$ sont remplacés par leurs approximations discrètes $V_h$ et $V_h^D$ définies par

$$
\begin{array}{rll}
V_h &=& \left\{ v_h \in \mathcal{C}^0(\overline \Omega); \; v_{|T}\in \mathbb{Q}_1(Q), \; \forall Q\in \mathcal{Q_h}; \; v_{h|\Gamma_D}=0 \right\},\\
V_h^D &=& \left\{ v_h \in \mathcal{C}^0(\overline \Omega); \; v_{|T}\in \mathbb{Q}_1(Q), \; \forall Q\in \mathcal{Q_h}; \; v_{h|\Gamma_D}=u_D \right\}.
\end{array}
$$

### Discrétisation éléments finis
La formulation variationnelle discrète s'écrit

Trouver $u_h\in V_h^D$ tel que $\forall v_h \in V_h$

$$
\sum_{Q\in \mathcal{Q}_h}\int_Q \epsilon(u_h):\mathbb{C}\epsilon(v_h)dx=\sum_{Q\in \mathcal{Q}_h}\int_Q f\cdot v_h dx + \sum_{E\subset \Gamma_N}\int_{\Gamma_N} g\cdot v_h ds,
$$

avec $u_h=(u_{1h},u_{2h})\in V_h$,

$$
u_{\alpha h}=\sum \varphi_j(x) u_{\alpha}^{j}, \; \alpha = 1,2.
$$

### Assemblage de la matrice de rigidité

La matrice élémentaire s'écrit

$$
\displaystyle{A_{ij}^e = \int_Q \epsilon(\varphi_i): \mathbb{C}\epsilon(\varphi_j) dx}
$$

### Notation de Voigt

En remarquant que le tenseur $\mathbb{C}$ est symétrique et $\epsilon_{ij} = \epsilon_{ji}$, $\sigma(u_h) = \mathbb{C}\epsilon(u_h)$ peut se réécrire

$$
\left(
\begin{array}{c}
\sigma_{11} \\
\sigma_{22} \\
\sigma_{12} 
\end{array}
\right)
=
C \gamma(u_h),
$$

avec

$$
C = \left(
\begin{array}{ccc}
\lambda+2\mu & \lambda & 0 \\
\lambda & \lambda+2\mu & 0 \\
0 & 0 & \mu
\end{array}
\right)
\; \text{et} \;
\gamma(u) = \left(
\begin{array}{c}
\epsilon_{11} \\
\epsilon_{22} \\
2\epsilon_{12} 
\end{array}
\right)=
\left(
\begin{array}{c}
\frac{\partial u_1}{\partial x} \\
\frac{\partial u_2}{\partial y} \\ 
\frac{\partial u_1}{\partial y}+\frac{\partial u_2}{\partial x} 
\end{array}
\right)
$$

### Fonctions de base

<table>
<tr>
<td><img src="./figures/element.png"></img></td>
<td>
$$
\begin{array}{rll}
\varphi_1(x,y) &=& \displaystyle{\frac{(x-h_x)(y-h_y)}{h_xh_y}}\\
\varphi_2(x,y) &=& \displaystyle{-\frac{x(y-h_y)}{h_xh_y}}\\
\varphi_3(x,y) &=& \displaystyle{\frac{xy}{h_xh_y}}\\
\varphi_4(x,y) &=& \displaystyle{-\frac{(x-h_x)y}{h_xh_y}}
\end{array}
$$
    </td>
</tr>
</table>

Soit $u = (u_1, u_2, \cdots, u_8)$ les valeurs de la solution $u_h$ aux noeuds du quadrange $Q$. Alors

$$
\gamma(u) = \left(
\begin{array}{cccccccc}
\varphi_{1,x} & 0 & \varphi_{2,x} & 0 & \varphi_{3,x} & 0 & \varphi_{4,x} & 0 \\
0 & \varphi_{1,y} & 0 & \varphi_{2,y} & 0 & \varphi_{3,y} & 0 & \varphi_{4,y} \\
\varphi_{1,y} & \varphi_{1,x} &\varphi_{2,y} & \varphi_{2,x} &\varphi_{3,y} & \varphi_{3,x} &\varphi_{4,y} & \varphi_{4,x} 
\end{array}
\right)
\left(
\begin{array}{c}
u_1 \\
\vdots \\
u_8
\end{array}
\right)
= R u
$$

Il s'en suit

$$
\epsilon(v_h) : \mathbb{C}\epsilon(u_h) = v^tR^tCRu.
$$

La matrice élémentaire peut donc se réécrire comme

$$
A^e = R^tCR.
$$

### Utilisation de sympy

In [ ]:
import sympy as sp

lamb, mu = sp.symbols("lambda, mu")
dim = 2

C = sp.zeros(3, 3)
C[:dim, :dim] = lamb*sp.ones(dim, dim) + 2*mu*sp.eye(dim)
C[-1, -1] = mu

In [ ]:
sp.pprint(C)

### Définition des fonctions de base

In [ ]:
x, y, hx, hy = sp.symbols("x, y, hx, hy ")

basis_functions = [ (x - hx)*(y - hy)/(hx*hy),
                          -x*(y - hy)/(hx*hy),
                                  x*y/(hx*hy),
                          -(x - hx)*y/(hx*hy)]

sp.pprint(basis_functions)

### Construction de R

In [ ]:
phi = basis_functions
R = sp.zeros(C.shape[0], 2*len(phi))

deriv = [x, y]

for j in range(dim):
    for i in range(len(phi)):
        R[j, dim*i + j] = phi[i].diff(deriv[j])

for i in range(len(phi)):
    R[-1, dim*i] = phi[i].diff(y)
    R[-1, dim*i + 1] = phi[i].diff(x)

In [ ]:
sp.pprint(R)

### Construction de $A^e$

In [ ]:
A = R.T*C*R

Ae = sp.zeros(dim*len(phi), dim*len(phi))
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        Ae[i, j] = sp.integrate(A[i, j], (y, 0., hy), (x, 0., hx)).expand()
        Ae[i, j].together().factor()

In [ ]:
sp.pprint(Ae)

### PETSc (Portable, Extensible Toolkit for Scientific Computation)

<center>
<img style="width:70%;" src="./figures/petsc_1.png"/>
</center>

### PETSc (Portable, Extensible Toolkit for Scientific Computation)

**Langages supportés**

- C/C++
- Fortran
- Python (`petsc4py`)

**Interface avec des bibliothèques externes**

- FFTW
- Hyper
- MUMPS
- ParMetis,
- SuperLU,
- UMFPACK,
- ...

### DM

Petsc offre deux structures de données pour gérer les maillages

- `DMDA` pour les maillages uniformes cartésiens,
- `DMPlex` pour les maillages non structurés.

### DMDA

In [21]:
%%px
from petsc4py import PETSc

nx = ny = 5

da = PETSc.DMDA().create([nx, ny], dof=2, stencil_width=1)
da.setUniformCoordinates(xmax=1, ymax=1)

### DMDA (4 sous domaines)
<center>
<img style="width:60%" src="./figures/DMDA.png"/>
</center>

### Récupérer $x_s$, $gx_s$, $x_e$, $gx_e$, ...

In [22]:
%%px
print(PETSc.COMM_WORLD.rank, da.getRanges(), da.getGhostRanges())

[stdout:0] 2 ((0, 3), (3, 5)) ((0, 4), (2, 5))
[stdout:1] 3 ((3, 5), (3, 5)) ((2, 5), (2, 5))
[stdout:2] 0 ((0, 3), (0, 3)) ((0, 4), (0, 4))
[stdout:3] 1 ((3, 5), (0, 3)) ((2, 5), (0, 4))


### Parcourir les éléments avec une numérotation locale

In [23]:
%%px
da.getElements()

Out[0:29]: 
array([[ 0,  1,  5,  4],
       [ 1,  2,  6,  5],
       [ 4,  5,  9,  8],
       [ 5,  6, 10,  9]], dtype=int32)

Out[1:29]: 
array([[0, 1, 4, 3],
       [1, 2, 5, 4],
       [3, 4, 7, 6],
       [4, 5, 8, 7]], dtype=int32)

Out[2:29]: 
array([[ 0,  1,  5,  4],
       [ 1,  2,  6,  5],
       [ 4,  5,  9,  8],
       [ 5,  6, 10,  9]], dtype=int32)

Out[3:29]: 
array([[0, 1, 4, 3],
       [1, 2, 5, 4],
       [3, 4, 7, 6],
       [4, 5, 8, 7]], dtype=int32)

### Matrice et vecteur à partir d'un DMDA

In [24]:
%%px 
A = da.createMatrix()
x = da.createGlobalVec()
xlocal = da.createLocalVec()

On peut toujours accéder aux éléments en utilisant la numérotation globale.

In [25]:
%px x[0] = 1

In [26]:
%px print(da.getVecArray(x)[:])

[stdout:0] 
[[[ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]]]
[stdout:1] 
[[[ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]]]
[stdout:2] 
[[[ 1.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]]
[stdout:3] 
[[[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]]


### Assemblage de la matrice

In [27]:
%%px
from elasticity.assembling import getMatElemElasticity, getIndices

h, lamb, mu = [.25, .25], .25, .1

Melem = getMatElemElasticity(h, lamb, mu)
A = da.createMatrix()
elem = da.getElements()

dof = da.getDof()
for e in elem:
    ind = getIndices(e, dof)
    A.setValuesLocal(ind, ind, Melem, PETSc.InsertMode.ADD_VALUES)
A.assemble()

### Création du KSP

In [28]:
%%px
ksp = PETSc.KSP().create()
ksp.setOperators(A)
ksp.setType("cg")
pc = ksp.getPC()
pc.setType("gamg")
ksp.setFromOptions()

### Exemple

In [29]:
import sys, petsc4py
petsc4py.init(sys.argv)
from petsc4py import PETSc
import numpy as np
from elasticity import *

def rhs(coords, rhs):
    rhs[..., 1] = -9.81

# set options
OptDB = PETSc.Options()
Lx = OptDB.getReal('Lx', 10)
Ly = OptDB.getReal('Ly', 1)
n  = OptDB.getInt('n', 16)
nx = OptDB.getInt('nx', Lx*n)
ny = OptDB.getInt('ny', Ly*n)

hx = Lx/(nx - 1)
hy = Ly/(ny - 1)

da = PETSc.DMDA().create([nx, ny], dof=2, stencil_width=1)
da.setUniformCoordinates(xmax=Lx, ymax=Ly)

# constant young modulus
E = 30000
# constant Poisson coefficient
nu = 0.4

lamb = (nu*E)/((1+nu)*(1-2*nu)) 
mu = .5*E/(1+nu)

x = da.createGlobalVec()
b = buildRHS(da, [hx, hy], rhs)
A = buildElasticityMatrix(da, [hx, hy], lamb, mu)
A.assemble()

bcApplyWest(da, A, b)

ksp = PETSc.KSP().create()
ksp.setOperators(A)
ksp.setFromOptions()

ksp.solve(b, x)

viewer = PETSc.Viewer().createVTK('solution_2d.vts', 'w', comm = PETSc.COMM_WORLD)
x.view(viewer)

In [18]:
! mpiexec -n 4 python demos/elasticity_2d.py -ksp_monitor -ksp_type gmres -pc_type gamg

  0 KSP Residual norm 3.445660734014e-01 
  1 KSP Residual norm 7.268982844720e-02 
  2 KSP Residual norm 7.098886477230e-02 
  3 KSP Residual norm 6.885586558112e-02 
  4 KSP Residual norm 6.867283608183e-02 
  5 KSP Residual norm 6.359484123651e-02 
  6 KSP Residual norm 4.172033470394e-02 
  7 KSP Residual norm 4.079709471894e-02 
  8 KSP Residual norm 4.078189287147e-02 
  9 KSP Residual norm 3.609454024623e-02 
 10 KSP Residual norm 3.581976582440e-02 
 11 KSP Residual norm 3.173352967053e-02 
 12 KSP Residual norm 3.137811911970e-02 
 13 KSP Residual norm 3.054147953655e-02 
 14 KSP Residual norm 2.710207001531e-02 
 15 KSP Residual norm 2.565197027242e-02 
 16 KSP Residual norm 2.503711986399e-02 
 17 KSP Residual norm 2.342554146776e-02 
 18 KSP Residual norm 2.000434711303e-02 
 19 KSP Residual norm 1.584962972383e-02 
 20 KSP Residual norm 1.216314487639e-02 
 21 KSP Residual norm 8.014364582359e-03 
 22 KSP Residual norm 4.116936200681e-03 
 23 KSP Residual norm 1.9073034067

In [32]:
def plot_solution(filename):
    import vtk
    reader = vtk.vtkXMLStructuredGridReader()
    reader.SetFileName(filename)
    reader.Update()

    nodes_array = reader.GetOutput().GetPoints().GetData() 
    data_arrays = reader.GetOutput().GetPointData()
    u = data_arrays.GetArray(0)
    v = data_arrays.GetArray(1)
    
    from vtk.util.numpy_support import vtk_to_numpy
    import numpy as np
    nodes = vtk_to_numpy(nodes_array)
    u = vtk_to_numpy(u)
    v = vtk_to_numpy(v)
    
    x = nodes[:, 0] + u
    y = nodes[:, 1] + v
    
    from bqplot import pyplot as plt

    plt.figure()
    plt.scatter(x, y, color=np.sqrt(u**2 + v**2), default_size=2)
    plt.show()

In [35]:
plot_solution('solution_2d.vts')

VBox(children=(Figure(axes=[ColorAxis(scale=ColorScale()), Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Scatter(color=array([  4.97852316e-08,   8.50702463e-03,   1.40437064e-02, ...,
         4.04863145e+00,   4.08266807e+00,   4.11670489e+00]), colors=['DeepSkyBlue'], default_size=2, interactions={'hover': 'tooltip'}, scales={'color': ColorScale(), 'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([  2.30120815e-08,   5.77230351e-02,   1.15747380e-01, ...,
         1.01453634e+01,   1.02082563e+01,   1.02711495e+01]), y=array([ -4.41476317e-08,  -6.75574462e-03,  -9.82082024e-03, ...,
        -3.03954137e+00,  -3.07365396e+00,  -3.10776546e+00]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), Toolbar(figure=Figure(axes=[ColorAxis(scale=ColorScale()), Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Scatter(color=array([  4.97852316e-08,   8.50702463e-03,   1.40437064e-02, ...,
         4.04863145e+00,   4.08266807e+00,   4.11670489e+00]), colors=['DeepSkyBlue'], default_size=2, interactions={'hover': 'tooltip'}, scales={'color': ColorScale(), 'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([  2.30120815e-08,   5.77230351e-02,   1.15747380e-01, ...,
         1.01453634e+01,   1.02082563e+01,   1.02711495e+01]), y=array([ -4.41476317e-08,  -6.75574462e-03,  -9.82082024e-03, ...,
        -3.03954137e+00,  -3.07365396e+00,  -3.10776546e+00]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)))))

### Construction d'un préconditionneur

On prend une partition sans recouvrement de $\Omega$.

$$
\left\{
\begin{array}{l}
\overline \Omega = \bigcup_{s=1}^N \overline \Omega_s \\
\Omega_s \cap \Omega_t = \emptyset; \; s \neq t,
\end{array}
\right.
$$

où $N$ est le nombre de sous domaines.

### Construction d'un préconditionneur

$$
M^{-1} = \sum_{s=1}^N R_s^TA_s^+R_s \; \text{où} \; A_s = D^{-1}_s A_{|\Omega_s} D^{-1}_s,
$$

où

$$
A_{|\Omega_s} = \int_{\Omega_s} \epsilon(v):\mathbb{C}\epsilon(u) dx \; + (BC \cap \Omega_s).
$$

$R_s$ est l'opérateur de restriction et $D_s$ une partition de l'unité de sorte que

$$
\sum_{s=1}^N R_s^TD_sR_s = I.
$$

### Projection A orthogonale sur $ker(Z^TA)$

On définit

$$
\Pi = I - Z(Z^TAZ)^{-1}Z^TA
$$

où $Z$ sont des vecteurs qui engendrent $\sum_{s=1}^N R_s^TKer(A_s)$.

Dans notre cas, $Ker(A_s) \subset D_s RBM$.

On a 

$$
\Pi^T A \Pi = A \Pi = \Pi^T A.
$$

### Construction de $R_s$ et de $D_s^{-1}$

<center>
<img style="width:70%" src="./figures/partition.png"/>
</center>

### Construction de $R_s$ et de $D_s^{-1}$

In [30]:
%%px
(xs, xe), (ys, ye) = da.getRanges()
(gxs, gxe), (gys, gye) = da.getGhostRanges()

Rlocal = da.createLocalVec()
Rlocal_a = da.getVecArray(Rlocal)
Rlocal_a[gxs:xe, gys:ye] = 1

# multiplicity
Dinv = da.createGlobalVec()
Dinvlocal = da.createLocalVec()
da.localToGlobal(Rlocal, Dinv, addv=PETSc.InsertMode.ADD_VALUES)
da.globalToLocal(Dinv, Dinvlocal)

### Construction de $\Pi$

In [ ]:
RBM = PETSc.NullSpace().createRigidBody(da.getCoordinates())
rbm_vecs = RBM.getVecs()

work1 = da.createLocalVec()
work2 = da.createLocalVec()

Z = []
for i in range(mpi.COMM_WORLD.size):
    for ivec, rbm_vec in enumerate(rbm_vecs):
        Z.append(da.createGlobalVec())
        work1.set(0)
        da.globalToLocal(rbm_vec, work2)
        if i == mpi.COMM_WORLD.rank:
            work1 = work2*Rlocal/Dinvlocal
        da.localToGlobal(work1, Z[-1], addv=PETSc.InsertMode.ADD_VALUES)

# BC
AZ = []
for vec in vecs:
    bcApplyWest_vec(da, Z)
    AZ.append(A*Z)

# orthonormalize
for i, z in enumerate(Z):
    alphas = []
    for az in AZ[:i]:
        alphas.append(z.dot(az))
    for alpha, zi in zip(alphas, Z[:i]):
        z.axpy(-alpha, zi)
    z.scale(1./np.sqrt(vec.dot(A*z)))
    AZ[i] = A*z

### Construction du préconditionneur

In [69]:
class ASM(object):
    def __init__(self, da_global, Dinv_global, Z, AZ, h, lamb, mu):
        self.da_global = da_global
        self.vecs = vecs
        self.Avecs = Avecs

        (xs, xe), (ys, ye) = self.da_global.getRanges()
        (gxs, gxe), (gys, gye) = self.da_global.getGhostRanges()
        self.block = (slice(gxs, xe), slice(gys, ye))

        self.da_local = PETSc.DMDA().create([xe-gxs, ye-gys], dof=2, 
                                            stencil_width=1,
                                            comm=PETSc.COMM_SELF)
        mx, my = self.da_local.getSizes()
        self.da_local.setUniformCoordinates(xmax=h[0]*mx, ymax=h[1]*my)

        A = buildElasticityMatrix(self.da_local, h, lamb, mu)
        A.assemble()

        Dinv = self.da_local.createGlobalVec()
        Dinvlocal_a = self.da_local.getVecArray(Dinv)
        Dinvlocal = self.da_global.createLocalVec()
        self.da_global.globalToLocal(Dinv_global, Dinvlocal)
        D_a = self.da_global.getVecArray(Dinvlocal)

        Dinvlocal_a[:, :] = Dinv_a[self.block]
        A.diagonalScale(Dinv, Dinv)

        if mpi.COMM_WORLD.rank == 0:
            bcApplyWestMat(self.da_local, A)

        self.nullspace = PETSc.NullSpace().createRigidBody(self.da_local.getCoordinates())
        u, v, r = self.nullspace.getVecs()
        u[:] /= Dinv[:]
        v[:] /= Dinv[:]
        r[:] /= Dinv[:]
        if mpi.COMM_WORLD.rank == 0:
            bcApplyWest_vec(self.da_local, u)
            bcApplyWest_vec(self.da_local, v)
            bcApplyWest_vec(self.da_local, r)

        A.setNullSpace(self.nullspace)

        # build local solvers
        self.ksp = PETSc.KSP().create()
        self.ksp.setOperators(A)
        self.ksp.setOptionsPrefix("myasm_")
        self.ksp.setType('cg')
        pc = self.ksp.getPC()
        pc.setType('none')
        self.ksp.setFromOptions()

        # Construct work arrays
        self.work_global = self.da_global.createLocalVec()
        self.workg_global = self.da_global.createGlobalVec()
        self.work1_local = self.da_local.createGlobalVec()
        self.work2_local = self.da_local.createGlobalVec()

    def mult(self, mat, x, y):
        self.work_global.set(0.)
        self.da_global.globalToLocal(x, self.work_global)

        work_global_a = self.da_global.getVecArray(self.work_global)

        work1_local_a = self.da_local.getVecArray(self.work1_local)
        work1_local_a[:, :] = work_global_a[self.block]

        self.ksp.solve(self.work1_local, self.work2_local)

        self.work_global.set(0.)
        sol_a = self.da_local.getVecArray(self.work2_local)

        work_global_a[self.block] = sol_a[:, :]
        self.da_global.localToGlobal(self.work_global, y, addv=PETSc.InsertMode.ADD_VALUES)

        # Apply projection
        self.workg_global.set(0)
        for z, Az in zip(self.Z, self.AZ):
            self.workg_global += Az.dot(y)*z
        
        y -= self.workg_global
        
class PCASM(object):
    def setUp(self, pc):
        B, self.P = pc.getOperators()

    def apply(self, pc, x, y):
        y.array = self.P*x        

### Résolution du système avec notre nouveau préconditionneur

In [ ]:
def rhs(coords, rhs):
    rhs[..., 1] = -9.81

OptDB = PETSc.Options()
Lx = OptDB.getInt('Lx', 10)
Ly = OptDB.getInt('Ly', 1)
n  = OptDB.getInt('n', 16)
nx = OptDB.getInt('nx', Lx*n)
ny = OptDB.getInt('ny', Ly*n)

hx = Lx/(nx - 1)
hy = Ly/(ny - 1)

da = PETSc.DMDA().create([nx, ny], dof=2, stencil_width=1)
da.setUniformCoordinates(xmax=Lx, ymax=Ly)

# constant young modulus
E = 30000
# constant Poisson coefficient
nu = 0.4

lamb = (nu*E)/((1+nu)*(1-2*nu)) 
mu = .5*E/(1+nu)

x = da.createGlobalVec()
b = buildRHS(da, [hx, hy], rhs)
A = buildElasticityMatrix(da, [hx, hy], lamb, mu)
A.assemble()

bcApplyWest(da, A, b)

# build nullspace and multiplicity
Dinv, Z, AZ = get_nullspace(da, A)

# set matrix of the preconditioner
asm = ASM(da, D, vecs, Avecs, [hx, hy], lamb, mu)
P = PETSc.Mat().createPython(
    [x.getSizes(), b.getSizes()], comm=da.comm)
P.setPythonContext(asm)
P.setUp()

# Set initial guess
ptilde = da.createGlobalVec()
for i in range(len(Z)):
    ptilde += Z[i].dot(b)*Z[i]
x = ptilde
bcApplyWest_vec(da, x)

ksp = PETSc.KSP().create()
ksp.setOperators(A, P)
ksp.setOptionsPrefix("elas_")
ksp.setType("cg")
ksp.setInitialGuessNonzero(True)
# set our own preconditioner
pc = ksp.pc
pc.setType(pc.Type.PYTHON)
pc.setPythonContext(PCASM())
ksp.setFromOptions()

ksp.solve(b, x)

viewer = PETSc.Viewer().createVTK('solution_2d_bdd.vts', 'w', comm = PETSc.COMM_WORLD)
x.view(viewer)

In [21]:
! mpiexec -n 4 python demos/schwarz.py -elas_ksp_monitor -elas_ksp_type cg

  Residual norms for elas_ solve.
  0 KSP Residual norm 1.029282029670e+02 
  1 KSP Residual norm 1.227135117725e-02 
  2 KSP Residual norm 1.118330669606e-03 
  3 KSP Residual norm 4.524011436624e-05 
  4 KSP Residual norm 1.351227085042e-05 


In [34]:
plot_solution('solution_2d_bdd.vts')

VBox(children=(Figure(axes=[ColorAxis(scale=ColorScale()), Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Scatter(color=array([ 0.        ,  0.00850706,  0.01404378, ...,  4.0486644 ,
        4.08270139,  4.11673857]), colors=['DeepSkyBlue'], default_size=2, interactions={'hover': 'tooltip'}, scales={'color': ColorScale(), 'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([  0.        ,   0.05772301,   0.11574732, ...,  10.14536626,
        10.20825913,  10.27115237]), y=array([ 0.        , -0.00675577, -0.00982087, ..., -3.03957421,
       -3.07368716, -3.10779903]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), Toolbar(figure=Figure(axes=[ColorAxis(scale=ColorScale()), Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Scatter(color=array([ 0.        ,  0.00850706,  0.01404378, ...,  4.0486644 ,
        4.08270139,  4.11673857]), colors=['DeepSkyBlue'], default_size=2, interactions={'hover': 'tooltip'}, scales={'color': ColorScale(), 'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([  0.        ,   0.05772301,   0.11574732, ...,  10.14536626,
        10.20825913,  10.27115237]), y=array([ 0.        , -0.00675577, -0.00982087, ..., -3.03957421,
       -3.07368716, -3.10779903]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)))))